In [ ]:
# Recreating Zach's prospector generation code but with my parameter space

In [1]:
from matplotlib.pyplot import *
from prospectFunctions import *
import time
from prospect.sources import CSPSpecBasis #, FastStepBasis
from prospect.models.templates import TemplateLibrary#, describe
from astropy.cosmology import Planck13
from prospect.models import SpecModel
from tqdm import tqdm
import itertools
import astropy.units as u
import astropy.constants as const

In [2]:
run_params = {}
run_params['zred'] = 3.548
run_params['mass'] = 1e8
run_params['add_duste'] = False
run_params['zcontinuous'] = 1

grid_ranges = {}
grid_ranges['logzsol'] = np.linspace(-1,.5,10)
grid_ranges['dust2'] = np.linspace(0,3,10)
grid_ranges['tage'] = np.linspace(7,9,10)

# a = 1.0 + run_params['zred']

# obs = getObs(**run_params)
# sps = getSps(**run_params)
# wspec, wphot = getWave(obs=obs,sps=sps,**run_params)

In [3]:
# ==================MAKE GRID OF PROSPECTOR GALAXIES======================

n_elem = 5 # number of elements for M*, SFR, z, 12+log(O/H). this is the "resolution"
n_param = 3 # number of parameters we'll vary

dust_arr = np.linspace(0, 3, n_elem)
logzsol_arr = np.linspace(-1, .5, n_elem)
tage_arr = np.linspace(7, 9, n_elem)

pros_mat = np.array(np.meshgrid(logzsol_arr, dust_arr, tage_arr)).T
pros_arr = pros_mat.reshape(int(np.prod(pros_mat.shape) / n_param), n_param)

print('Prospector matrix shape:', pros_arr.shape)

Prospector matrix shape: (125, 3)


In [ ]:
# generate empty arrays to fill
logzsol_pros, dust_pros, tage_pros = pros_arr[:, 0], pros_arr[:, 1], pros_arr[:, 2]

In [4]:
start = time.time()

sps = CSPSpecBasis(zcontinuous=1)
# filters = observate.load_filters(['jwst_f150w', 'jwst_f444w']) # we'll want F150W and F444W
# see link below to add additional photometry
# https://github.com/bd-j/sedpy/blob/main/sedpy/data/filters/README.md
# obs = dict(wave_effective=[1494.4, 4378.7], spectrum=None, unc=None,
#            maggies=np.ones(len(filters))*1e-10, maggies_unc=np.ones(len(filters))*1e-10, filters=filters)

obs = getObs(**run_params)

end = time.time()
print(f'{(end-start):.2f}')

wspec, wphot = getWave(obs=obs,sps=sps,**run_params)

12.53


In [5]:
data_grid_params = np.empty((len(grid_ranges['logzsol']),len(grid_ranges['dust2']),len(grid_ranges['tage']),5))
data_grid_spec = np.empty((len(grid_ranges['logzsol']),len(grid_ranges['dust2']),len(grid_ranges['tage']),len(wspec)))
data_grid_phot = np.empty((len(grid_ranges['logzsol']),len(grid_ranges['dust2']),len(grid_ranges['tage']),len(wphot)))

In [11]:
start = time.time()

for i_logz,i_dust,i_tage in tqdm(itertools.product(range(len(grid_ranges['logzsol'])),range(len(grid_ranges['dust2'])),range(len(grid_ranges['tage'])))):
    
    logz = grid_ranges['logzsol'][i_logz]
    dust = grid_ranges['dust2'][i_dust]
    tage = grid_ranges['tage'][i_tage]

    run_params['tage'] = tage
    run_params['dust2'] = dust
    run_params['logzsol'] = logz

    model = getModel(**run_params)
    
    init_spec, init_phot, init_mfrac = model.predict(model.theta, obs=obs, sps=sps)

    flux_density_phot = []
    for wave,mag in zip(wphot,init_phot):
        jy = mag*3631.0 * u.Jy
        flux = jy.to(u.erg / u.cm**2 / u.s / u.AA, equivalencies=u.spectral_density(wave*u.AA))
        flux_density_phot.append(flux.value*10**20)

    flux_density_spec = []
    for wave,mag in zip(wspec,init_spec):
        jy = mag*3631.0 * u.Jy
        flux = jy.to(u.erg / u.cm**2 / u.s / u.AA, equivalencies=u.spectral_density(wave*u.AA))
        flux_density_spec.append(flux.value*10**20)

    data_grid_params[i_logz, i_dust, i_tage, :] = [run_params['zred'], run_params['mass'], logz, dust, tage]
    data_grid_spec[i_logz, i_dust, i_tage, :] = init_spec # flux_density_spec
    data_grid_phot[i_logz, i_dust, i_tage, :] = init_phot # flux_density_phot

end = time.time()

136it [12:04,  5.33s/it]


KeyboardInterrupt: 

In [10]:
# 1000 iterations ^ in 3:02 (without converting units)
# 1000 iterations ^ 5.33s/it (with converting units) D:

In [ ]:
# START LOOP
print('Generating prospector models')
for pp in tqdm(range(data_grid_params.shape[0])):

    run_params
    
    # model = getModel(**run_params)

    # try:
    #     mass_fracs[pp] = sps.ssp.stellar_mass / sps.ssp.formed_mass
    #     # prospector is weird. we give it the total mass, but only get the stellar mass out after the fact.
    # except: mass_fracs[pp] = np.nan

    # ssfr = 10 ** (lsfr_pros[pp]) / ((10 ** mass_pros[pp]) * mass_fracs[pp])
    # model_params['gas_logu']['init'] = get_logu(np.log10(ssfr))
    
    # model = SpecModel(model_params)

    # # current_parameters = ",".join([f"{p}={v}" for p, v in zip(model.free_params, model.theta)])
    # spec, phot, mfrac = model.predict(model.theta, obs=obs, sps=sps)

    # emlines = (model._eline_lum * u.L_sun / u.M_sun * (10 ** mass_pros[pp] * u.M_sun))
    # emlines = emlines / (4 * np.pi * Planck13.luminosity_distance(z_pros[pp]) ** 2)
    # emlines = emlines.to(u.erg / u.s / u.cm ** 2).value
    # # https://github.com/cconroy20/fsps/blob/master/data/emlines_info.dat
    # has[pp], hbs[pp], oiiis[pp], siis0[pp], siis1[pp] = emlines[74], emlines[59], emlines[62], emlines[77], emlines[78]

    # f150w_pros[pp], f444w_pros[pp] = phot[0], phot[1] 

# else: 
#     has[pp], hbs[pp], oiiis[pp], siis0[pp], siis1[pp] = np.nan, np.nan, np.nan, np.nan, np.nan
#     f150w_pros[pp], f444w_pros[pp] = np.nan, np.nan

In [ ]:
model = getModel(**run_params)

start = time.time()
spec, phot, mfrac = model.predict(model.theta, obs, sps)
end = time.time()

print(end-start)

In [ ]:
# TemplateLibrary.show_contents()
# Probably will use TemplateLibrary['ssp']

# TemplateLibrary['ssp']: Basic set of (free) parameters for a delta function SFH.

# TemplateLibrary['parametric_sfh']: A "delay-tau" star formation history (SFH) is a model that describes how 
# the rate of star formation in a galaxy changes over time, specifically with a delayed exponential decline.

# model_params['mass']['isfree'] = False
# model_params['logzsol']['isfree'] = False
# model_params['dust2']['isfree'] = False
# model_params['tage']['isfree'] = False

print(TemplateLibrary['ssp'].keys())
TemplateLibrary['ssp']['tage']

In [ ]:
zred = run_params['zred']

model_params = {'agebins':{'init':None}}

t_now = Planck13.lookback_time(zred).value
model_params['agebins']['init'] = [[0,                      np.log10((t_now-0.1)*1e9)], 
                                [np.log10((t_now-0.1)*1e9), np.log10( t_now     *1e9)]] # units of log(years)
print(model_params['agebins']['init'])
# model_params['logsfr_ratios']['init'] = get_lsfr_ratio(mass_pros[pp], z_pros[pp], lsfr_pros[pp])
# model = SpecModel(model_params)

In [ ]:
model = getModel(**run_params)
model

In [ ]:
model = getModel(**run_params)
model

In [6]:
def getModel(mass=None, zred=None, logzsol=None, tage=None, dust2=None, **extras):
    """Build a prospect.models.SpecModel object

    :param mass: (optional, default:None)
        If given, produce spectra for this mass. Otherwise the mass will
        be 1e8 solar masses.

    :param zred: (optional, default: None)
        If given, produce spectra and observed frame photometry appropriate
        for this redshift. Otherwise the redshift will be zero.

    :param logzsol: (optional, default: None)
        If given, fix the model metallicity (:math: `log(Z/Z_sun)`) to the given value.
        Otherwise the metallicity will be set to -0.5.
        
    :param tage: (optional, default: None)
        If given, produce spectra and model photometry appropriate for
        this galactic age. Otherwise the age will be set to 13. Gyrs.

    :param dust2: (optional, default: None)
        If given, produce spectra that are appropriate for provided dust
        attenuation. Otherwise attenuation will be set to 0.6.

    :returns model:
        An instance of prospect.models.SedModel
    """
    from prospect.models import SpecModel
    from prospect.models.templates import TemplateLibrary

    model_params = TemplateLibrary['ssp']

    # Change `isfree` so that all parameters that will be kept track of are identified 
    # in the `model` object as `free_params`
    model_params['zred']['isfree'] = True

    if zred is None:
        model_params['zred']['init'] = 0.0
    else:
        model_params['zred']['init'] = zred


    # for wave,mag in zip(wphot,init_phot):
    #     jy = mag*3631.0 * u.Jy
    #     flux = jy.to(u.erg / u.cm**2 / u.s / u.AA, equivalencies=u.spectral_density(wave*u.AA))
    #     flux_density_phot.append(flux.value*10**20)

    # flux_density_spec = []
    # for wave,mag in zip(wspec,init_spec):
    #     jy = mag*3631.0 * u.Jy
    #     flux = jy.to(u.erg / u.cm**2 / u.s / u.AA, equivalencies=u.spectral_density(wave*u.AA))
    #     flux_density_spec.append(flux.value*10**20)    if mass is not None:
        model_params['mass']['init'] = mass

    if logzsol is not None:
        model_params['logzsol']['init'] = logzsol

    if tage is None:
        model_params['tage']['init'] = 13.
    else:
        model_params['tage']['init'] = tage

    if dust2 is not None:
        model_params['dust2']['init'] = dust2

    model = SpecModel(model_params)

    return model
    
start = time.time()
test_model = getModel(**run_params)
end = time.time()
print(end-start)

0.0009908676147460938


----------

In [ ]:
print(f"FastStepBasis: {(12.17+12.06)/2}, CSPSpecBasis: {(12.84+12.91)/2}")
print(f"Difference: {(12.875 - 12.115):.3f}")